In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.utils.data import download_file
import os
from PIL import Image
from IPython.display import display
from matplotlib.patches import Circle
from astropy.wcs import WCS 
from astropy.coordinates import SkyCoord
import astropy.units as u
#import regions
from matplotlib.patches import Circle
from astropy.nddata import Cutout2D
import astropy.table as Table
from astropy.table import Column
from astroquery.simbad import Simbad
from astropy.io import ascii
from IPython.display import clear_output
from astromy_ds9 import ds9_norm
import openpyxl

In [ ]:
# Inicializa los arreglos
region_definitions = []

x_coords = []
y_coords = []
shape_labels = []
descriptions = []

In [ ]:
#10350_0145 Parameters

ds9_10350_0020 ={

    # Set your specific high and low limits for each channel
    "red_low_limit" :3.56574 ,   # Adjust as needed
    "red_high_limit" :13.3056 ,  # Adjust as needed
    "green_low_limit" :-0.171222 ,  # Adjust as needed
    "green_high_limit" :3.17238  ,  # Adjust as needed
    "blue_low_limit" : -1.71875 ,    # Adjust as needed
    "blue_high_limit" : 3.48553,   # Adjust as needed

    # Set contrast and bias parameters for each channel
    "red_contrast" : 2.01493 ,
    "red_bias" : 0.171642,
    "green_contrast" : 0.895522,
    "green_bias" : 0.440298,
    "blue_contrast" : 1.52985 ,
    "blue_bias" : 0.406716 ,
}

# Set your specific high and low limits for each channel
red_low_limit = 3.56574  # Adjust as needed
red_high_limit = 13.3056 # Adjust as needed
green_low_limit = -0.171222 # Adjust as needed
green_high_limit = 3.17238 # Adjust as needed
blue_low_limit = -1.71875    # Adjust as needed
blue_high_limit = 3.48553  # Adjust as needed

# Set contrast and bias parameters for each channel
red_contrast = 2.01493
red_bias = 0.171642
green_contrast = 0.895522
green_bias = 0.440298
blue_contrast = 1.52985
blue_bias = 0.406716

In [ ]:
#10350_0145 Parameters

ds9_10350_0145 ={

    # Set your specific high and low limits for each channel
    "red_low_limit" : 1.06747,   # Adjust as needed
    "red_high_limit" : 13.34,  # Adjust as needed
    "green_low_limit" : -3.08926,  # Adjust as needed
    "green_high_limit" : 4.38916,  # Adjust as needed
    "blue_low_limit" : -0.270054,    # Adjust as needed
    "blue_high_limit" : 5.66525,   # Adjust as needed

    # Set contrast and bias parameters for each channel
    "red_contrast" : 5.10638,
    "red_bias" : 0.621094,
    "green_contrast" : 3.78251,
    "green_bias" : 0.66875,
    "blue_contrast" : 4.4208,
    "blue_bias" : 0.28359,
}

# Set your specific high and low limits for each channel
red_low_limit = 0.308013   # Adjust as needed
red_high_limit = 7.65212  # Adjust as needed
green_low_limit = -2.84365  # Adjust as needed
green_high_limit = 3.72721 # Adjust as needed
blue_low_limit = -1.07343    # Adjust as needed
blue_high_limit = 2.45755  # Adjust as needed

# Set contrast and bias parameters for each channel
red_contrast = 3.39552
red_bias = 0.869403
green_contrast = 2.72388
green_bias = 0.761194
blue_contrast = 2.53731
blue_bias = 0.63806

In [ ]:
#10350_0270 Parameters

ds9_10350_0270 ={

    # Set your specific high and low limits for each channel
    "red_low_limit" : 2.20949 ,   # Adjust as needed
    "red_high_limit" : 14.2415 ,  # Adjust as needed
    "green_low_limit" : -0.421766 ,  # Adjust as needed
    "green_high_limit" : 4.74487 ,  # Adjust as needed
    "blue_low_limit" : -0.687285,    # Adjust as needed
    "blue_high_limit" : 4.19035,   # Adjust as needed

    # Set contrast and bias parameters for each channel
    "red_contrast" : 5.93284,
    "red_bias" : 0.410448,
    "green_contrast" :5.37313 ,
    "green_bias" : 0.380597,
    "blue_contrast" : 7.91045,
    "blue_bias" : 0.466418,
}

# Set your specific high and low limits for each channel
red_low_limit = 2.20949   # Adjust as needed
red_high_limit = 14.2415  # Adjust as needed
green_low_limit = -0.421766# Adjust as needed
green_high_limit = 4.74487 # Adjust as needed
blue_low_limit =  -0.687285 # Adjust as needed
blue_high_limit =  4.19035# Adjust as needed

# Set contrast and bias parameters for each channel
red_contrast = 5.93284
red_bias = 0.410448
green_contrast = 5.37313
green_bias = 0.380597
blue_contrast = 7.91045
blue_bias = 0.466418



####OTHER SETTING PARAMS####
# Set your specific high and low limits for each channel
red_low_limit = -0.575399   # Adjust as needed
red_high_limit = 9.42872  # Adjust as needed
green_low_limit = -0.421766# Adjust as needed
green_high_limit = 1.46396 # Adjust as needed
blue_low_limit =  -1.90669 # Adjust as needed
blue_high_limit =  5.40976# Adjust as needed

# Set contrast and bias parameters for each channel
red_contrast = 4.24966
red_bias = 0.628726
green_contrast = 1
green_bias = 0.5
blue_contrast = 4.20152
blue_bias = 0.468835

In [ ]:
#10350_0145 Parameters

ds9_10550_0020 ={

    # Set your specific high and low limits for each channel
    "red_low_limit" :2.70244 ,   # Adjust as needed
    "red_high_limit" :12.4065 ,  # Adjust as needed
    "green_low_limit" :-1.04001,  # Adjust as needed
    "green_high_limit" :3.78033  ,  # Adjust as needed
    "blue_low_limit" : 0.189677 ,    # Adjust as needed
    "blue_high_limit" :3.0474,   # Adjust as needed

    # Set contrast and bias parameters for each channel
    "red_contrast" : 3.14039 ,
    "red_bias" : 0.5,
    "green_contrast" : 2.62357,
    "green_bias" : 0.619241,
    "blue_contrast" : 1.02662 ,
    "blue_bias" : 0.331978 ,
}

# Set your specific high and low limits for each channel
red_low_limit = 2.70244  # Adjust as needed
red_high_limit = 12.4065 # Adjust as needed
green_low_limit = -1.04001 # Adjust as needed
green_high_limit = 3.78033 # Adjust as needed
blue_low_limit = 0.189677    # Adjust as needed
blue_high_limit = 3.0474  # Adjust as needed

# Set contrast and bias parameters for each channel
red_contrast = 3.14039
red_bias = 0.5
green_contrast = 2.62357
green_bias = 0.619241
blue_contrast = 1.02662
blue_bias = 0.331978

In [ ]:
# Open FITS file
path = "10550_0020/"
hdu = fits.open(path + "10550_0020_ds9.fits")
wcs = WCS(hdu[0].header)

# Extract RGB data
RedData = hdu[1].data
GreenData = hdu[2].data
BlueData = hdu[3].data

# Set your specific high and low limits for each channel
red_low_limit = 2.70244  # Adjust as needed
red_high_limit = 12.4065 # Adjust as needed
green_low_limit = -1.04001 # Adjust as needed
green_high_limit = 3.78033 # Adjust as needed
blue_low_limit = 0.189677    # Adjust as needed
blue_high_limit = 3.0474  # Adjust as needed

# Set contrast and bias parameters for each channel
red_contrast = 3.14039
red_bias = 0.5
green_contrast = 2.62357
green_bias = 0.619241
blue_contrast = 1.02662
blue_bias = 0.331978


def update_image(RedData, GreenData, BlueData):
    # Apply scaling to each channel to emphasize peaks and reduce noise
    Rednorm = ds9_norm(vmin=red_low_limit, vmax=red_high_limit, bias=red_bias, contrast=red_contrast, stretch='sqrt')(RedData)
    Greennorm = ds9_norm(vmin=green_low_limit, vmax=green_high_limit, bias=green_bias, contrast=green_contrast, stretch='sqrt')(GreenData)
    Bluenorm = ds9_norm(vmin=blue_low_limit, vmax=blue_high_limit, bias=blue_bias, contrast=blue_contrast, stretch='sqrt')(BlueData)

    # Stack the normalized channels
    rgb_image = np.dstack((Rednorm, Greennorm, Bluenorm))

    # Display the RGB image
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb_image, origin='lower')
    plt.show()
    
    return Rednorm, Greennorm, Bluenorm


new_RedScaled, new_GreenScaled, new_BlueScaled = update_image(RedData, GreenData, BlueData)

In [ ]:
#Funcion myRegion
def myRegion(x_coord, y_coord, image, wcs, filtro, half_size=100):
    
    # Ensure the provided coordinates are in pixel units
    int_x_coord = int(x_coord)
    int_y_coord = int(y_coord)

    # Define the coordinates of the center of the region
    center_x = half_size
    center_y = half_size

    # Radius of the circle that surrounds the region
    radius = 20

    # Create a figure and axis
    px = 1 / plt.rcParams["figure.dpi"] # Pixel in inches.
    print(plt.rcParams["figure.dpi"])
    fig, ax = plt.subplots(figsize=(256 * px, 256 * px), subplot_kw={'projection': wcs})
    fig.subplots_adjust(top=1, bottom=0, left=0, right=1, wspace=0, hspace=0)
    plt.margins(0, 0)
    ax.set_axis_off()

    # Extract the region using WCS information from the red channel
    cutout = Cutout2D(image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)

    # Display the region extracted with WCS
    ax.imshow(cutout.data, origin='lower')

    # Draw the circle around the region
    # circle = Circle((center_x, center_y), radius, fill=False, color='blue', lw=2)
    # ax.add_patch(circle)

    # Disable axis labels and ticks
    ax.coords.grid(False)
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Display the plot
    print(filtro)
    plt.show()
    

In [ ]:
  #Funcion overlayFilters (funcion para hacer un overlay de los 3 filtro RGB)
def overlayFilters(x_coord, y_coord, red_image, green_image, blue_image, wcs, filtro, half_size=100,):
    # Ensure the provided coordinates are in pixel units
    int_x_coord = int(x_coord)
    int_y_coord = int(y_coord)

    # Define the coordinates of the center of the region
    center_x = half_size
    center_y = half_size

    # Radius of the circle that surrounds the region
    radius = 20

    # Create a figure and axis
    px = 1 / plt.rcParams["figure.dpi"] # Pixel in inches.
    print(plt.rcParams["figure.dpi"])
    fig, ax = plt.subplots(figsize=(256 * px, 256 * px), subplot_kw={'projection': wcs})
    fig.subplots_adjust(top=1, bottom=0, left=0, right=1, wspace=0, hspace=0)
    plt.margins(0, 0)
    ax.set_axis_off()

    # Extract the regions using WCS information from the different filters
    red_cutout = Cutout2D(red_image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)
    green_cutout = Cutout2D(green_image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)
    blue_cutout = Cutout2D(blue_image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)

    # Combine the filters into a composite RGB image
    composite_image = np.dstack((red_cutout.data, green_cutout.data, blue_cutout.data))

    # Display the composite image with WCS
    ax.imshow(composite_image, origin='lower')

    # Draw the circle around the region
    # circle = Circle((center_x, center_y), radius, fill=False, color='blue', lw=2)
    # ax.add_patch(circle)

    # Disable axis labels and ticks
    ax.coords.grid(False)
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Display the plot
    print(filtro)
    #plt.show()
    
    return fig

In [ ]:
#Escribe archivos y directorios necesarios para encontrar la informacion
path ="10550_0020/"

# Define the filename for the table
filename = 'region_info_10550_0020.csv'

# Sube region file in physical coordinates
region_file_physical = path + 'fisicas_10550_0020.reg'
# Sube region file in galactic coordinates
region_file_galactic = path + 'galacticas_10550_0020.reg'  

In [ ]:
### Verificar si tabla existe o no
import openpyxl.drawing
import openpyxl.drawing.image


try:
    data_table = ascii.read(filename)
except FileNotFoundError:
    data_table = Table(names=('x_coord', 'y_coord', 'shape_label', 'description'), 
                       dtype=('f8', 'f8', 'S20', 'S100'))
    
####################################################################################################################

with open(region_file_physical, 'r') as f_physical:
    lines_physical = f_physical.readlines()

with open(region_file_galactic, 'r') as f_galactic:
    lines_galactic = f_galactic.readlines()

# Initialize Excel file.
wb = openpyxl.Workbook()
ws = wb.worksheets[0]
count = 1
tile = 1

# Iterate over each line in the physical coordinate file
for line_physical, line_galactic in zip(lines_physical, lines_galactic):
    # Remove leading/trailing whitespace and skip empty lines
    line_physical = line_physical.strip()
    line_galactic = line_galactic.strip()
    
    # Extract the physical and Galactic coordinates
    physical_parts = line_physical.split()
    galactic_parts = line_galactic.split()
    
    physical_x_str, physical_y_str = physical_parts[0], physical_parts[1]
    x = float(physical_x_str)
    y = float(physical_y_str)

    galactic_x_str, galactic_y_str = galactic_parts[0], galactic_parts[1]
    galactic_x = float(galactic_x_str)
    galactic_y = float(galactic_y_str)
    
    # Check if the data already exists in the table
    if any((data_table['x_coord'] == galactic_x) & (data_table['y_coord'] == galactic_y)):
        # If data exists, skip
        print("Data already exists in the table for Galactic coordinates:", galactic_x_str, galactic_y_str)
        continue
      
    else:

        # Print the Galactic coordinates
        print("Region Galactic:")
        print(galactic_x_str, galactic_y_str)
        print()

        # Functions to display images
        figure = overlayFilters(x, y, new_RedScaled, new_GreenScaled, new_BlueScaled, wcs, "RGB", half_size=50)

        # Save figure into image.
        imgname = "cuts/plot" + str(count) + ".png"
        plt.savefig(imgname)

        myRegion(x, y, new_RedScaled, wcs, "Rojo-8 micras", half_size=50)
        myRegion(x, y, new_GreenScaled, wcs, "Verde-4.6 micras", half_size=50)
        myRegion(x, y, new_BlueScaled, wcs, "Azul-3.5 micras", half_size=50)

        # Write figure into Excel spreadsheet.
        img = openpyxl.drawing.image.Image(imgname)
        ws.add_image(img, "A" + str(tile))
        wb.save("output.xlsx")
    
        # #Input validation
        # while True:
        #     try:
        #         # Menu
        #         print("Elige forma:")
        #         print("1. Alargada\n2. Redonda\n3. Difusa\n4. Espiral\n5. Otra\n")

        #         num = int(input())
        #         break  # Si la conversión a entero es exitosa, salimos del ciclo while

        #     except ValueError:
        #         # Si la conversión falla, mostramos un mensaje de error
        #         print("Error: Ingresa un número entero válido.")

        # if num == 1:
        #     figure.savefig(path + "Alargadas/" + galactic_x_str + "," + galactic_y_str + "_Alargada.jpeg")
        #     morph = "Alargada"
        # elif num == 2:
        #     figure.savefig(path + "Redondas/" + galactic_x_str + "," + galactic_y_str +"_Redonda.jpeg")
        #     morph = "Redonda"
        # elif num == 3:
        #     figure.savefig(path + "Difusas/" + galactic_x_str + "," + galactic_y_str + "_Difusa.jpeg")
        #     morph = "Difusa"
        # elif num == 4:
        #     figure.savefig(path + "Espirales/" + galactic_x_str + "," + galactic_y_str + "_Espiral.jpeg")
        #     morph = "Espiral"
        # else:
        #     figure.savefig(path + "Otras/" + galactic_x_str + "," + galactic_y_str + "_Otra.jpeg")
        #     morph = "Otra"

        # Cierra la foto
        plt.close(figure)
        count += 1
        tile += 13
    
        #Para tabular
        # description = input("Descripcion de la fuente: ")

        # x_coords.append(galactic_x_str)
        # y_coords.append(galactic_y_str)
        # shape_labels.append(morph)
        # descriptions.append(description)
    
        # # Create the columns
        # x_col = Column(x_coords, name='x_coord')
        # y_col = Column(y_coords, name='y_coord')
        # shape_label_col = Column(shape_labels, name='shape_label')
        # description_col = Column(descriptions, name='description')

        # # Create an Astropy Table from the columns
        # region_table = Table.Table([x_col, y_col, shape_label_col, description_col])


        #  # Save the table to a file (e.g., CSV)
        # region_table.write('region_info_10550_0020.csv', format='csv', overwrite=True)

        # Clear the output cell in Jupyter Notebook
        clear_output(wait=False)